In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
clients = pd.read_csv('/content/drive/MyDrive/SwiftTraq/Portfolio/015_MSP_Tickets_Analysis/data/Clients.csv')
agents = pd.read_csv('/content/drive/MyDrive/SwiftTraq/Portfolio/015_MSP_Tickets_Analysis/data/Agents.csv')
tickets = pd.read_csv('/content/drive/MyDrive/SwiftTraq/Portfolio/015_MSP_Tickets_Analysis/data/Tickets.csv')

In [ ]:
clients.head()

,client_id,client_name,industry,location
0,C001,"Reese, Brown and Carpenter",Education,North Chasefort
1,C002,"Dunn, Gomez and Harmon",Technology,Fordhaven
2,C003,Herrera Inc,Retail,Erikahaven
3,C004,"Wheeler, Beasley and Knight",Technology,North Louisborough
4,C005,Molina and Sons,Technology,Clarkport


In [ ]:
agents.head()

,agent_id,agent_name,team,seniority_level
0,A001,Lisa Dennis,Tier 3,Mid
1,A002,Michelle Henderson,Tier 3,Senior
2,A003,Michael Lloyd,Tier 1,Junior
3,A004,Michelle Allen,Tier 3,Mid
4,A005,Kimberly Harvey,Tier 3,Junior


In [ ]:
tickets.head()

,ticket_id,client_id,department,opened_date,resolved_date,sla_hours,priority,agent_id
0,T0001,C048,Security,2024-11-11 04:33:31,2024-11-13 03:25:15.845927,48,Medium,A024
1,T0002,C022,Security,2025-03-10 02:48:01,2025-03-12 07:25:35.783302,48,Medium,A003
2,T0003,C001,IT Support,2024-12-06 15:05:10,2024-12-06 21:11:59.949618,72,Medium,A014
3,T0004,C015,Networking,2025-02-18 18:30:09,2025-02-20 03:10:53.785603,48,Low,A013
4,T0005,C049,Cloud Services,2025-01-21 21:41:52,2025-01-22 08:30:42.952956,72,Medium,A004


In [ ]:
clients.isnull().sum()

,0
client_id,0
client_name,0
industry,0
location,0


In [ ]:
agents.isnull().sum()

,0
agent_id,0
agent_name,0
team,0
seniority_level,0


In [ ]:
tickets.isnull().sum()

,0
ticket_id,0
client_id,0
department,0
opened_date,0
resolved_date,0
sla_hours,0
priority,0
agent_id,0


In [ ]:
# Check for outliers -- > 168 hrs[week]

len(tickets[tickets['sla_hours'] > 168])

0

In [ ]:
# convert dates to actual dates
tickets['opened_date'] = pd.to_datetime(tickets['opened_date'])
tickets['resolved_date'] = pd.to_datetime(tickets['resolved_date'])

In [ ]:
tickets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   ticket_id      1000 non-null   object        
 1   client_id      1000 non-null   object        
 2   department     1000 non-null   object        
 3   opened_date    1000 non-null   datetime64[ns]
 4   resolved_date  1000 non-null   datetime64[ns]
 5   sla_hours      1000 non-null   int64         
 6   priority       1000 non-null   object        
 7   agent_id       1000 non-null   object        
dtypes: datetime64[ns](2), int64(1), object(5)
memory usage: 62.6+ KB


🛠️ Feature Engineering

In [ ]:
tickets['resolution_hours'] = (tickets['resolved_date'] - tickets['opened_date']).dt.total_seconds() / 3600

In [ ]:
tickets['resolution_hours'].head()

,resolution_hours
0,46.862457
1,52.626329
2,6.113875
3,32.679107
4,10.814154


In [ ]:
# SLA Met
tickets['sla_met'] = (tickets['resolution_hours'] <= tickets['sla_hours']).astype(int)

In [ ]:
# Week / Month opened
tickets['opened_week'] = tickets['opened_date'].dt.isocalendar().week
tickets['month_opened'] = tickets['opened_date'].dt.to_period('M')

In [ ]:
# Tickets Age
now = datetime.now()
tickets['ticket_age_days'] = ( now - tickets['opened_date']).dt.days

In [ ]:
# Priority Encoding
#tickets['priority'].unique()
tickets['priority_level'] = tickets['priority'].map({'Low': 1, 'Medium': 2, 'High': 3})

In [ ]:
# merge -- clients, agents
merged_df = tickets.merge(clients, on='client_id', how='left').merge(agents, on='agent_id', how='left')

In [ ]:
merged_df.head()

,ticket_id,client_id,department,opened_date,resolved_date,sla_hours,priority,agent_id,resolution_hours,sla_met,opened_week,month_opened,ticket_age_days,priority_level,client_name,industry,location,agent_name,team,seniority_level
0,T0001,C048,Security,2024-11-11 04:33:31,2024-11-13 03:25:15.845927,48,Medium,A024,46.862457,1,46,2024-11,172,2,Lewis-Cole,Technology,North Patrickville,Kaitlyn Patrick,Tier 3,Junior
1,T0002,C022,Security,2025-03-10 02:48:01,2025-03-12 07:25:35.783302,48,Medium,A003,52.626329,0,11,2025-03,53,2,Ochoa and Sons,Technology,North John,Michael Lloyd,Tier 1,Junior
2,T0003,C001,IT Support,2024-12-06 15:05:10,2024-12-06 21:11:59.949618,72,Medium,A014,6.113875,1,49,2024-12,147,2,"Reese, Brown and Carpenter",Education,North Chasefort,Courtney Thompson,Tier 2,Junior
3,T0004,C015,Networking,2025-02-18 18:30:09,2025-02-20 03:10:53.785603,48,Low,A013,32.679107,1,8,2025-02,73,1,Watson LLC,Education,Josephhaven,Tyler Rivera,Tier 2,Senior
4,T0005,C049,Cloud Services,2025-01-21 21:41:52,2025-01-22 08:30:42.952956,72,Medium,A004,10.814154,1,4,2025-01,101,2,Thomas Ltd,Finance,Cabreraport,Michelle Allen,Tier 3,Mid


💹 Aggregated KPIs

In [ ]:
# SLA performance -- departmentwise
sla_by_dept = merged_df.groupby('department')['sla_met'].agg(['count', 'mean'])

(sla_by_dept.rename(columns={'count': 'ticket_volume', 'mean': 'sla_met_rate'})
.sort_values(by='sla_met_rate', ascending=False)
.reset_index() )

,department,ticket_volume,sla_met_rate
0,Networking,254,0.657480
1,Security,260,0.650000
2,Cloud Services,229,0.637555
3,IT Support,257,0.603113


In [ ]:
# Resolution time by agent
# avg >> median -- means some tickets take much longer/problematic to the individual
agent_perf = (merged_df.groupby('agent_name')['resolution_hours']
 .agg(['count', 'mean', 'median'])
 .rename(columns={'count': 'tickets_handled', 'mean': 'avg_resolution_hrs', 'median': 'median_resolution_hrs'})
 .sort_values(by='median_resolution_hrs', ascending=True)
 .reset_index()
 )

agent_perf.head()

,agent_name,tickets_handled,avg_resolution_hrs,median_resolution_hrs
0,Stefanie Palmer,46,30.863124,23.064860
1,Heather Eaton,29,36.098008,25.143515
2,Kimberly Harvey,37,37.256035,25.805611
3,Lisa Dennis,47,40.756879,27.027221
4,David Perkins,34,42.013453,27.637717


In [ ]:
# Ticket Distribution by Industry & Priority
industry_priority = merged_df.groupby(['industry', 'priority']).size().reset_index(name='ticket_count')
industry_priority.head()

,industry,priority,ticket_count
0,Education,High,56
1,Education,Low,88
2,Education,Medium,126
3,Finance,High,43
4,Finance,Low,69


💾 Save Cleaned data

In [ ]:
path = "/content/drive/MyDrive/SwiftTraq/Portfolio/015_MSP_Tickets_Analysis/data/"

merged_df.to_csv(path+'cleaned_ticket_data.csv', index=False)
sla_by_dept.to_csv(path+'sla_by_dept.csv', index=False)
agent_perf.to_csv(path+'agent_perf.csv', index=False)
industry_priority.to_csv(path+'industry_priority.csv', index=False)

print('✅ Datasets saved successfully!')

✅ Datasets saved successfully!
